# Лабораторная работа №2

Описание

В вашем распоряжении данные сервиса Яндекс Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктах за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. Ваша задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность.
По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма.

Описание данных

- airports_nearest — расстояние до ближайшего аэропорта в метрах (м)
- balcony — число балконов
- ceiling_height — высота потолков (м)
- cityCenters_nearest — расстояние до центра города (м)
- days_exposition — сколько дней было размещено объявление (от публикации до снятия)
- first_day_exposition — дата публикации
- floor — этаж
- floors_total — всего этажей в доме
- is_apartment — апартаменты (булев тип)
- kitchen_area — площадь кухни в квадратных метрах (м²)
- last_price — цена на момент снятия с публикации
- living_area — жилая площадь в квадратных метрах (м²)
- locality_name — название населённого пункта
- open_plan — свободная планировка (булев тип)
- parks_around3000 — число парков в радиусе 3 км
- parks_nearest — расстояние до ближайшего парка (м)
- ponds_around3000 — число водоёмов в радиусе 3 км
- ponds_nearest — расстояние до ближайшего водоёма (м)
- rooms — число комнат
- studio — квартира-студия (булев тип)
- total_area — площадь квартиры в квадратных метрах (м²)
- total_images — число фотографий квартиры в объявлении

Для начала импортируем все библиотеки и считываем файл с датасетом. При этом сталкиваемся с проблемой - pandas всё видит как одну колонку. В нашем случае разделители у датасета не стандартная запятая, а табуляция, поэтому чтобы pandas считал всё правильно мы дописываем параметр sep='\t' при чтении файла.

In [203]:
def climbStairs(n: int, counter = 0) -> int:
    if n >= 2:
        climbStairs(n - 2, counter + 1)
        print(counter, n)
        climbStairs(n - 1, counter + 1)
        print(counter, n)
    elif n == 1:
        climbStairs(n - 1, counter + 1)
        print(counter, n)
    else:
        return counter
    
print(climbStairs(n=4))

1 2
2 1
1 2
0 4
2 1
1 3
2 2
3 1
2 2
1 3
0 4
None


In [163]:
import pandas as pd
import numpy as np

data = pd.read_csv('real_estate_data.csv', sep='\t')

In [164]:
pd.set_option('display.max_columns', None) # Команда, чтобы строки целиком показывались при использовании display

In [165]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23699 entries, 0 to 23698
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   total_images          23699 non-null  int64  
 1   last_price            23699 non-null  float64
 2   total_area            23699 non-null  float64
 3   first_day_exposition  23699 non-null  object 
 4   rooms                 23699 non-null  int64  
 5   ceiling_height        14504 non-null  float64
 6   floors_total          23613 non-null  float64
 7   living_area           21796 non-null  float64
 8   floor                 23699 non-null  int64  
 9   is_apartment          2775 non-null   object 
 10  studio                23699 non-null  bool   
 11  open_plan             23699 non-null  bool   
 12  kitchen_area          21421 non-null  float64
 13  balcony               12180 non-null  float64
 14  locality_name         23650 non-null  object 
 15  airports_nearest   

### Обработка пропусков

Можно заметить, что в датасете есть пропуски в столбцах: ceiling_height, living_area, is_apartment, kitchen_area, balcony, airports_nearest, cityCenters_nearest, parks_around3000, parks_nearest, ponds_around3000, ponds_nearest, days_exposition, floors_total, locality_name. 

Для некоторых пропущенных значений можно предположить логичную замену. Например, если человек не указал число балконов — скорее всего, их нет. Такие пропуски допустимо заменить на 0.

In [166]:
data['balcony'] = data['balcony'].fillna(0)

Теперь проанализируем информацию о близлежащих объектах. 

Начнём с ponds и для начала посмотрим какие значения появляются у ponds_around3000 и ponds_nearest.

In [167]:
data['ponds_around3000'].value_counts(dropna=False)

ponds_around3000
0.0    9071
1.0    5717
NaN    5518
2.0    1892
3.0    1501
Name: count, dtype: int64

In [168]:
data['ponds_nearest'].value_counts(dropna=False)

ponds_nearest
NaN       14589
427.0        70
454.0        41
153.0        40
433.0        39
          ...  
986.0         1
131.0         1
725.0         1
40.0          1
1134.0        1
Name: count, Length: 1097, dtype: int64

Можно заметить, что в ponds_around3000 помимо нулевых значений, есть NaN, но говорить о том, что NaN можно заменить на 0, как в случае с balcony пока рано. В случае с ponds_nearest можно заметить, что есть 14589 строк с пропуском этого значения. Сейчас требуется взглянуть на строки где ponds_around3000 или ponds_nearest равны NaN или ponds_around3000 равен 0.

In [169]:
data[(data['ponds_around3000'] == 0) | (data['ponds_around3000'].isna()) | (data['ponds_nearest'].isna())]

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
1,7,3350000.0,40.40,2018-12-04T00:00:00,1,NaN,11.0,18.60,1,NaN,False,False,11.00,2.0,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
5,10,2890000.0,30.40,2018-09-10T00:00:00,1,NaN,12.0,14.40,5,NaN,False,False,9.10,0.0,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55.0
6,6,3700000.0,37.30,2017-11-02T00:00:00,1,NaN,26.0,10.60,6,NaN,False,False,14.40,1.0,посёлок Парголово,52996.0,19143.0,0.0,NaN,0.0,NaN,155.0
7,5,7915000.0,71.60,2019-04-18T00:00:00,2,NaN,24.0,NaN,22,NaN,False,False,18.90,2.0,Санкт-Петербург,23982.0,11634.0,0.0,NaN,0.0,NaN,NaN
8,20,2900000.0,33.16,2018-05-23T00:00:00,1,NaN,27.0,15.43,26,NaN,False,False,8.81,0.0,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,189.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23690,3,5500000.0,52.00,2018-07-19T00:00:00,2,NaN,5.0,31.00,2,NaN,False,False,6.00,0.0,Санкт-Петербург,20151.0,6263.0,1.0,300.0,0.0,NaN,15.0
23692,2,1350000.0,30.00,2017-07-07T00:00:00,1,NaN,5.0,17.50,4,NaN,False,False,6.00,0.0,Тихвин,NaN,NaN,NaN,NaN,NaN,NaN,413.0
23695,14,3100000.0,59.00,2018-01-15T00:00:00,3,NaN,5.0,38.00,4,NaN,False,False,8.50,0.0,Тосно,NaN,NaN,NaN,NaN,NaN,NaN,45.0
23696,18,2500000.0,56.70,2018-02-11T00:00:00,2,NaN,3.0,29.70,1,NaN,False,False,NaN,0.0,село Рождествено,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Итого мы получили то самое число 14589. Это значит, что ponds_nearest равен NaN только там. где ponds_around3000 равен NaN или 0. Для ponds_nearest логично иметь NaN, которое буквально означает отсутствие расстояния, поэтому его обрабатывать никак не требуется. Если говорить о том, стоит ли заполнять пропуски у ponds_around3000 нулями, то скорее всего это плохая идея. Как минимум можно взглянуть на водоёмы у Всеволожска:

In [170]:
data.loc[data['locality_name'] == 'Всеволожск']['ponds_around3000'].value_counts(dropna=False)

ponds_around3000
NaN    398
Name: count, dtype: int64

В этом городе они однозначно есть, и шанс что каждое из 400 объявлений были дальше чем на 3 км скорее всего близко к невозможному. Скорее всего в данном случае NaN означает именно отсутствие данных по этим объявлениям (то есть автоматическая картографическая система не смогла рассчитать эти признаки для объявлений), поэтому заполнять их нулями - неправильно.

Также мы поступим и с парками. для начала посмотрим какие значения появляются у parks_around3000 и parks_nearest.

In [171]:
data['parks_around3000'].value_counts(dropna=False)

parks_around3000
0.0    10106
1.0     5681
NaN     5518
2.0     1747
3.0      647
Name: count, dtype: int64

In [172]:
data['parks_nearest'].value_counts(dropna=False)

parks_nearest
NaN       15620
441.0        67
173.0        41
392.0        41
456.0        40
          ...  
794.0         1
760.0         1
1167.0        1
4.0           1
2984.0        1
Name: count, Length: 996, dtype: int64

Можно также заметить, что в parks_around3000 помимо нулевых значений, есть NaN. В случае с parks_nearest можно заметить, что есть 15620 строк с пропуском этого значения. Сейчас требуется аналогично взглянуть на строки где parks_around3000 или parks_nearest равны NaN или parks_around3000 равен 0.

In [173]:
data[(data['parks_around3000'] == 0) | (data['parks_around3000'].isna()) | (data['parks_nearest'].isna())]

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
1,7,3350000.0,40.40,2018-12-04T00:00:00,1,NaN,11.0,18.60,1,NaN,False,False,11.00,2.0,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
5,10,2890000.0,30.40,2018-09-10T00:00:00,1,NaN,12.0,14.40,5,NaN,False,False,9.10,0.0,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55.0
6,6,3700000.0,37.30,2017-11-02T00:00:00,1,NaN,26.0,10.60,6,NaN,False,False,14.40,1.0,посёлок Парголово,52996.0,19143.0,0.0,NaN,0.0,NaN,155.0
7,5,7915000.0,71.60,2019-04-18T00:00:00,2,NaN,24.0,NaN,22,NaN,False,False,18.90,2.0,Санкт-Петербург,23982.0,11634.0,0.0,NaN,0.0,NaN,NaN
8,20,2900000.0,33.16,2018-05-23T00:00:00,1,NaN,27.0,15.43,26,NaN,False,False,8.81,0.0,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,189.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23691,11,9470000.0,72.90,2016-10-13T00:00:00,2,2.75,25.0,40.30,7,NaN,False,False,10.60,1.0,Санкт-Петербург,19424.0,4489.0,0.0,NaN,1.0,806.0,519.0
23692,2,1350000.0,30.00,2017-07-07T00:00:00,1,NaN,5.0,17.50,4,NaN,False,False,6.00,0.0,Тихвин,NaN,NaN,NaN,NaN,NaN,NaN,413.0
23695,14,3100000.0,59.00,2018-01-15T00:00:00,3,NaN,5.0,38.00,4,NaN,False,False,8.50,0.0,Тосно,NaN,NaN,NaN,NaN,NaN,NaN,45.0
23696,18,2500000.0,56.70,2018-02-11T00:00:00,2,NaN,3.0,29.70,1,NaN,False,False,NaN,0.0,село Рождествено,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Можно заметить, что в этот раз мы получили не то же число, что и NaN у parks_nearest. Давайте взглянем на значения parks_nearest для того же условия, потому что если прибавить количество нулей к отстутствующим значениям у parks_nearest мы как раз получим 15624. Следовательно эти 4 строки появляются из-за parks_nearest.

In [174]:
data[(data['parks_around3000'] == 0) | (data['parks_around3000'].isna()) | (data['parks_nearest'].isna())]['parks_nearest'].value_counts(dropna=False)

parks_nearest
NaN       15620
3190.0        2
3064.0        1
3013.0        1
Name: count, dtype: int64

Можно увидеть те самые 4 значения, которые причем больше 3000. Это не противоречит ничему, поэтому логика обработки пропусков остаётся аналогичной обработке пропусков у водоёмов. Можно даже проверить снова город Всеволожоск на парки, в котором их достаточно много и убедиться, что заполнять пропуски тоже будет ошибкой.

In [175]:
data.loc[data['locality_name'] == 'Всеволожск']['parks_around3000'].value_counts(dropna=False)

parks_around3000
NaN    398
Name: count, dtype: int64

Из признаков с пропусками, которые заполняются автоматически системой, остались только airports_nearest и cityCenters_nearest.

In [176]:
data['airports_nearest'].value_counts(dropna=False).sort_index()

airports_nearest
0.0           1
6450.0        2
6914.0        1
6949.0        1
6989.0        6
           ... 
84006.0       1
84665.0       1
84853.0       1
84869.0       1
NaN        5542
Name: count, Length: 8276, dtype: int64

In [177]:
data['cityCenters_nearest'].value_counts(dropna=False)

cityCenters_nearest
NaN        5519
8460.0       61
20802.0      32
10720.0      30
20444.0      27
           ... 
11965.0       1
6228.0        1
2345.0        1
17882.0       1
16671.0       1
Name: count, Length: 7643, dtype: int64

In [178]:
data[(data['cityCenters_nearest'].isna()) & (data['airports_nearest'].isna())]

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
5,10,2890000.0,30.40,2018-09-10T00:00:00,1,NaN,12.0,14.40,5,NaN,False,False,9.10,0.0,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55.0
8,20,2900000.0,33.16,2018-05-23T00:00:00,1,NaN,27.0,15.43,26,NaN,False,False,8.81,0.0,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,189.0
12,10,3890000.0,54.00,2016-06-30T00:00:00,2,NaN,5.0,30.00,5,NaN,False,False,9.00,0.0,Сертолово,NaN,NaN,NaN,NaN,NaN,NaN,90.0
22,20,5000000.0,58.00,2017-04-24T00:00:00,2,2.75,25.0,30.00,15,NaN,False,False,11.00,2.0,деревня Кудрово,NaN,NaN,NaN,NaN,NaN,NaN,60.0
30,12,2200000.0,32.80,2018-02-19T00:00:00,1,NaN,9.0,NaN,2,NaN,False,False,NaN,0.0,Коммунар,NaN,NaN,NaN,NaN,NaN,NaN,63.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23683,16,2100000.0,62.80,2018-09-18T00:00:00,4,2.50,5.0,45.50,3,NaN,False,False,5.50,0.0,посёлок Дзержинского,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23692,2,1350000.0,30.00,2017-07-07T00:00:00,1,NaN,5.0,17.50,4,NaN,False,False,6.00,0.0,Тихвин,NaN,NaN,NaN,NaN,NaN,NaN,413.0
23695,14,3100000.0,59.00,2018-01-15T00:00:00,3,NaN,5.0,38.00,4,NaN,False,False,8.50,0.0,Тосно,NaN,NaN,NaN,NaN,NaN,NaN,45.0
23696,18,2500000.0,56.70,2018-02-11T00:00:00,2,NaN,3.0,29.70,1,NaN,False,False,NaN,0.0,село Рождествено,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Исходя из этого, можно заметить, что большая часть пропусков у этих столбцов связана с одними и теми же строками. Эти строки - 23 процента от общей выборки. Просто так без необходимости пробовать их заполнять не имеет особо смысла - так можно исказить данные.

Теперь перейдём к пропускам столбцов, значения которых устанавливает сам пользователь. Для начала стоит отметить, что в большинстве случае скорее всего - отсутствие данных в этих столбцах значит то, что пользователь либо забыл указать, либо не знает что указывать.

Разберём столбцы living_area и kitchen_area (8% и 10% пропущенных значений соответственно). В целом такое количество пропусков допустимо заполнить выражением (total_area текущей квартиры * средняя доля нужной части квартиры)

In [179]:
# средние доли
living_ratio = (data['living_area'] / data['total_area']).mean()
kitchen_ratio = (data['kitchen_area'] / data['total_area']).mean()
print(living_ratio, kitchen_ratio)

0.5647655216993234 0.1873547387717406


In [180]:
# заполнение пропусков
data.loc[data['living_area'].isna(), 'living_area'] = data['total_area'] * living_ratio
data.loc[data['kitchen_area'].isna(), 'kitchen_area'] = data['total_area'] * kitchen_ratio

Далее столбец floors_total. Имеет 86 пропусков. В целом можно удалить эти строки и потери не будут существеные, но я сейчас придерживаюсь стратегии сохранить максимальное количество строк, поэтому просто оставлю их нетронутыми. Заполнить эти пропуски можно, но это будут предположения по типу: если этаж 8, то велика вероятность, что всего их 9, или же писать в количество этаж тот этаж, что и в столбце floor. Я считаю на данный момент особой нужды в этом нет.

In [181]:
data.loc[data['floors_total'].isna()]

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
186,12,11640000.0,65.2,2018-10-02T00:00:00,2,NaN,NaN,30.800000,4,NaN,False,False,12.000000,0.0,Санкт-Петербург,39197.0,12373.0,1.0,123.0,0.0,NaN,49.0
237,4,2438033.0,28.1,2016-11-23T00:00:00,1,NaN,NaN,20.750000,1,NaN,False,False,5.264668,0.0,Санкт-Петербург,22041.0,17369.0,0.0,NaN,1.0,374.0,251.0
457,4,9788348.0,70.8,2015-08-01T00:00:00,2,NaN,NaN,38.400000,12,NaN,False,False,10.630000,0.0,Санкт-Петербург,37364.0,8322.0,2.0,309.0,2.0,706.0,727.0
671,4,6051191.0,93.6,2017-04-06T00:00:00,3,NaN,NaN,47.100000,8,NaN,False,False,16.800000,0.0,Санкт-Петербург,22041.0,17369.0,0.0,NaN,1.0,374.0,123.0
1757,5,3600000.0,39.0,2017-04-22T00:00:00,1,NaN,NaN,22.025855,9,NaN,False,False,7.306835,0.0,Санкт-Петербург,22735.0,11618.0,1.0,835.0,1.0,652.0,77.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22542,5,8500000.0,63.5,2017-05-24T00:00:00,2,2.8,NaN,35.862611,3,NaN,False,False,11.897026,0.0,Санкт-Петербург,51340.0,15363.0,0.0,NaN,1.0,853.0,512.0
22656,4,4574160.0,64.5,2017-04-02T00:00:00,2,NaN,NaN,31.700000,20,NaN,False,False,14.400000,0.0,Санкт-Петербург,22041.0,17369.0,0.0,NaN,1.0,374.0,127.0
22808,0,14569263.0,110.4,2016-11-20T00:00:00,3,NaN,NaN,45.380000,6,NaN,False,False,23.420000,0.0,Санкт-Петербург,19095.0,4529.0,0.0,NaN,0.0,NaN,260.0
23590,0,21187872.0,123.3,2017-04-25T00:00:00,3,NaN,NaN,50.400000,18,NaN,False,False,23.600000,0.0,Санкт-Петербург,19095.0,4529.0,0.0,NaN,0.0,NaN,104.0


Что касатеся locality_name, то тут всего 49 пропусков, поэтому эти строки целесообразно будет просто удалить. Поскольку мы даже не можем до конца быть уверенными, что это объявления точно из Санкт-Петербурга или из ближайших к нему населенных пунктов.

In [182]:
data.loc[data['locality_name'].isna()]

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
1097,3,8600000.0,81.70,2016-04-15T00:00:00,3,3.55,5.0,50.800000,2,NaN,False,False,8.800000,0.0,NaN,23478.0,4258.0,0.0,NaN,0.0,NaN,147.0
2033,6,5398000.0,80.00,2017-05-30T00:00:00,3,NaN,4.0,42.600000,2,NaN,False,False,18.600000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0
2603,20,3351765.0,42.70,2015-09-20T00:00:00,1,NaN,24.0,15.600000,3,NaN,False,False,10.700000,0.0,NaN,22041.0,17369.0,0.0,NaN,1.0,374.0,276.0
2632,2,5130593.0,62.40,2015-10-11T00:00:00,2,NaN,24.0,33.100000,21,NaN,False,False,8.200000,0.0,NaN,22041.0,17369.0,0.0,NaN,1.0,374.0,256.0
3574,10,4200000.0,46.50,2016-05-28T00:00:00,2,NaN,5.0,30.800000,5,NaN,False,False,6.500000,0.0,NaN,27419.0,8127.0,0.0,NaN,1.0,603.0,45.0
4151,17,17600000.0,89.50,2014-12-09T00:00:00,2,3.00,8.0,39.620000,7,NaN,False,False,13.380000,0.0,NaN,25054.0,3902.0,1.0,485.0,3.0,722.0,869.0
4189,7,9200000.0,80.00,2015-12-10T00:00:00,3,4.00,4.0,52.300000,3,False,False,False,10.400000,0.0,NaN,21774.0,3039.0,1.0,690.0,1.0,953.0,223.0
4670,1,5500000.0,83.00,2015-08-14T00:00:00,3,NaN,7.0,46.875538,6,NaN,False,False,15.550443,0.0,NaN,26534.0,5382.0,1.0,446.0,1.0,376.0,350.0
5343,19,13540000.0,85.50,2016-01-20T00:00:00,3,NaN,7.0,59.100000,5,False,False,False,8.300000,4.0,NaN,10556.0,9538.0,1.0,67.0,0.0,NaN,303.0
5707,7,3700000.0,30.00,2016-04-29T00:00:00,1,NaN,24.0,20.000000,23,NaN,False,False,5.620642,0.0,NaN,21460.0,16788.0,0.0,NaN,1.0,170.0,49.0


In [183]:
data = data.dropna(subset=['locality_name'])

- Столбец ceiling_height имеет 39% пропусков, заполнение медианными значениями также может привести к искажению данных.
- Столбец is_apartment имеет 88% пропусков, ситуация с этим столбцом ещё хуже, возможно для дальнейшей работы с этими данными его будет смысл удалить, т.к. данных просто мало одтосительно других столбцов. Заполнить его исходя чего-либо невозможно, т.к. ни с чем особо не коррелирует. Поэтому пропуски оставляем.
- Столбец days_exposition означает сколько дней было размещено объявление (от публикации до снятия), поэтому пропуски в нём точно никак не заполнить, данные в нём либо по ошибке в системе не появились, либо объявление у которого days_exposition равен NaN всё ещё активно.

Итого у нас получился датафрейм с такими строками:

In [184]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23650 entries, 0 to 23698
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   total_images          23650 non-null  int64  
 1   last_price            23650 non-null  float64
 2   total_area            23650 non-null  float64
 3   first_day_exposition  23650 non-null  object 
 4   rooms                 23650 non-null  int64  
 5   ceiling_height        14490 non-null  float64
 6   floors_total          23565 non-null  float64
 7   living_area           23650 non-null  float64
 8   floor                 23650 non-null  int64  
 9   is_apartment          2760 non-null   object 
 10  studio                23650 non-null  bool   
 11  open_plan             23650 non-null  bool   
 12  kitchen_area          23650 non-null  float64
 13  balcony               23650 non-null  float64
 14  locality_name         23650 non-null  object 
 15  airports_nearest      18